In [ ]:
from pymilvus import MilvusClient
client = MilvusClient(uri="db/sba_1c_database_milvus.db", local=True)
print(client.list_collections())

['risk_chunks', 'sba_1c_database_chunks']


In [3]:
from pymilvus import MilvusClient
client = MilvusClient(uri="db/tanezumab_milvus.db", local=True)
print(client.list_collections())

['risk_chunks', 'tanezumab_chunks']


In [4]:
from pymilvus import MilvusClient
client = MilvusClient(uri="data/NCT02709486_failed_simple.db", local=True)
print(client.list_databases)

<bound method MilvusClient.list_databases of <pymilvus.milvus_client.milvus_client.MilvusClient object at 0x128529790>>


In [5]:
import sqlite3

def checking_sqlite(sqlite_path):
    conn = sqlite3.connect(sqlite_path)
    cur = conn.cursor()
    # find user tables
    cur.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
    )
    tables = [r[0] for r in cur.fetchall()]
    if not tables:
        raise RuntimeError("No user tables found in SQLite DB")
    table = tables[0]
    cur.execute(f"PRAGMA table_info({table});")
    cols = cur.fetchall()  # (cid, name, type, notnull, dflt, pk)
    conn.close()

    # id column: pk or first integer
    id_col = next(
        (n for (_, n, t, _, _, pk) in cols if pk == 1),
        next((n for (_, n, t, _, _, _) in cols if "INT" in t.upper()), cols[0][1])
    )
    # text column: first TEXT-like
    text_col = next(
        (n for (_, n, t, _, _, _) in cols
         if any(x in t.upper() for x in ("CHAR", "CLOB", "TEXT"))),
        None
    )
    if not text_col:
        raise RuntimeError("No TEXT column found.")
    # metadata = all others
    meta_cols = [n for (_, n, _, _, _, _) in cols if n not in (id_col, text_col)]
    return table, id_col, text_col, meta_cols


In [7]:
checking_sqlite("data/NCT02709486_failed_simple.db")

('elements',
 'is_continuation',
 'type',
 ['element_id',
  'text',
  'text_as_html',
  'languages',
  'filename',
  'filetype',
  'data_source'])